In [117]:
# import rpy2.robjects as ro
# from rpy2.robjects import pandas2ri
# pandas2ri.activate()
# readRDS = ro.r['readRDS']
# df_from_r = readRDS('exgr_test.rds')
# df_python = pandas2ri.rpy2py_dataframe(df_from_r)
import pyreadr
import pandas as pd
rRead = pyreadr.read_r('exgr_test.rds')

In [118]:
df_python = pd.DataFrame(rRead[None])
df_python

,transcript_id,transcript_name,sequence_names,start,end,width,strand,exon_id,exon_name,rank
0,1,ENST00000456328.2,chr1,11869,12227,359,+,1,ENSE00002234944.1,1
1,1,ENST00000456328.2,chr1,12613,12721,109,+,5,ENSE00003582793.1,2
2,1,ENST00000456328.2,chr1,13221,14409,1189,+,8,ENSE00002312635.1,3
3,2,ENST00000450305.2,chr1,12010,12057,48,+,2,ENSE00001948541.1,1
4,2,ENST00000450305.2,chr1,12179,12227,49,+,3,ENSE00001671638.2,2
...,...,...,...,...,...,...,...,...,...,...
1460981,234481,ENST00000387409.1,chrM,5826,5891,66,-,760545,ENSE00001544488.1,1
1460982,234482,ENST00000387416.2,chrM,7446,7514,69,-,760546,ENSE00001544487.2,1
1460983,234483,ENST00000361681.2,chrM,14149,14673,525,-,760547,ENSE00001434974.2,1
1460984,234484,ENST00000387459.1,chrM,14674,14742,69,-,760548,ENSE00001544476.1,1


In [119]:
# Task 1: How many unique transcripts are there? 
df_python['transcript_id'].nunique()

234485

In [120]:
# Task 2: How many unique Exons are there?
df_python['exon_id'].nunique()

760549

In [121]:
# What is the average length of an exon? What is the median length?
# Average length of exon:
df_python['width'].mean()

262.9930635885628

In [122]:
# Median of length
df_python['width'].median()

130.0

In [ ]:
# We don't know if the data set was grouped and ordered by transcript_id and rank, so do it before operation
df_python = df_python.sort_values(by=['transcript_id', 'rank'])

In [133]:
# Task 4: Calculate the length of intron
# split the dataset into 2 sets, based on strand 
positive_strand = df_python['strand'] == '+'
negative_strand = df_python['strand'] == '-'


In [ ]:
# Calculate length of intron
# for positive strand: intron_length = nth start - (n-1)th end
df_python.loc[positive_strand, 'intron_length'] = df_python.loc[positive_strand, 'start'] - df_python.loc[positive_strand, 'end'].shift(1)

# for negative strand: intron_length = nth start - (n+1)th end
df_python.loc[negative_strand, 'intron_length'] = df_python.loc[negative_strand, 'start'] - df_python.loc[negative_strand, 'end'].shift(-1)


In [ ]:
# Set the first intron_length of each transcript to zero
# For positive strand: the first exon has the lowest rank, so get the index of first line of each transcript and transfer the correspond intron_length value to 0
Positive_Head_Idx = df_python[positive_strand].groupby('transcript_id').head(1).index
df_python.loc[Positive_Head_Idx, 'intron_length'] = 0

# For negative strand: the first exon has the highest rank, so get the index of last line of each transcript and transfer the correspond intron_length value to 0
Negative_Tail_Idx = df_python[negative_strand].groupby('transcript_id').tail(1).index
df_python.loc[Negative_Tail_Idx, 'intron_length'] = 0
